In [1]:
import json

In [2]:
import pandas as pd

In [3]:
    FILE_NAME = "14.03.19-14:16"
    source_comments = pd.read_csv(f'./wall/girl_wall/comments/{FILE_NAME}.csv', index_col=0)
    source_answers = pd.read_csv(f'./wall/girl_wall/answers/{FILE_NAME}.csv', index_col=0)
    source_posts = pd.read_csv(f'./wall/girl_wall/posts/{FILE_NAME}.csv', index_col=0)
    source_answers = source_answers.loc[:, ~source_answers.columns.str.contains('^Unnamed')]
    source_comments = source_comments.loc[:, ~source_comments.columns.str.contains('^Unnamed')]
    source_posts = source_posts.loc[:, ~source_posts.columns.str.contains('^Unnamed')]
    source_comments.replace({pd.np.NaN: None}, inplace=True)
    source_answers.replace({pd.np.NaN: None}, inplace=True)
    source_posts.replace({pd.np.NaN: None}, inplace=True)
    source_comments.drop_duplicates(inplace=True)
    source_answers.drop_duplicates(inplace=True)
    source_posts.drop_duplicates(inplace=True)
    source_comments.sort_values(by=['post_id', 'owner_id', 'date'], inplace=True)
    source_answers.sort_values(by=['post_id', 'owner_id', 'date'], inplace=True)

In [4]:
import scripts.wall_answers_to_dialogs as ans2dialog

In [5]:
[i for i in source_posts]

['date',
 'from_id',
 'num_comments',
 'owner_id',
 'post_id',
 'reply_owner_id',
 'reply_post_id',
 'text']

In [6]:
result = ans2dialog.get_dialogs_from_answers(source_answers, source_comments, None)

In [7]:
with open('./result.txt', 'w') as f:
    for num, dialog in enumerate(result):
        f.write(str(num)+" "+'\n')
        for _, row in pd.concat(dialog).iterrows():
            f.write(str(row['text'])+'\n')
        f.write('\n')

In [9]:
post_result = ans2dialog.post_process_dialogs_from_answers(result)

In [16]:
pd_result = list(map(lambda x: pd.concat(x), result))

In [17]:
type(pd_result[0])

pandas.core.frame.DataFrame

In [18]:
set(map(lambda x: len(set(x['from_id'])), post_result))

{1, 2, 9}

In [20]:
set(map(lambda x: len(set(x['from_id'])), pd_result))

{1, 2, 11}

In [44]:
only_2_persons = list(filter(lambda x: len(set(x['from_id'])) == 2, post_result))

In [51]:
import scripts.dialogs_to_json as d2j

In [52]:
seriz = d2j.dump_dialogs(only_2_persons,
                              "./filtered_users/21.02.19-12:08_girl.json",
                              "./filtered_users/21.02.19-12:08_pop.json",
                              source_posts)

In [53]:
seriz[0]

{'post_id': 244,
 'wall_owner_id': 1595309,
 'post_num_comments': 20,
 'post_text': '[id148888304|Елена Куликова]  [id263634|Мари Фомченко]  [id10656177|Ирина Мурсалимова]  [id139142982|Вероника Баглаева]  [id1424187|Алина Оганессян]  [id7067417|Света Антонова]  [id22484451|Татьяна Козлова]  [id1850347|Юлия Царёва] [id148888304|Елена Куликова]',
 'post_date': 1542727649,
 'dialog': [{'replica': {'comment_id': 245,
    'text': 'Спасибо!!! Даже не верится... так давно это было)',
    'date': 1542727975,
    'mess_from_id': 1424187,
    'reply_to_user': None,
    'type_author': 'other',
    'idx_speaker': 0}},
  {'replica': {'comment_id': 246,
    'text': '[id1424187|Алина], я тут на эти фотки наткнулась и не смогла удержаться.',
    'date': 1542728259,
    'mess_from_id': 1595309,
    'reply_to_user': None,
    'type_author': 'girl',
    'idx_speaker': 1}},
  {'replica': {'comment_id': 247,
    'text': '[id1424187|Алина], вообще-то я думала Вы меня прибьете.😜',
    'date': 1542728358,
  

In [58]:
import codecs
import json

with codecs.open('./result_json/dialogs_post_processed_27032019_1800.json', 'w', encoding='utf-8') as f:
    json.dump(seriz, f, ensure_ascii=False, indent=4)